In [1]:
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
import pymysql
pymysql.install_as_MySQLdb()

## Store files into dataframes
* Convert Date columns to same format for consistency
* Convert null values to zeros (e.g. when there is no markdown, populate 0 for the discount)
* Convert IsHoliday from bool to int format
* Delete extra columns (where applicable)

In [2]:
# setting variable to reference the csv file
features_file = "features_data-1.csv"
# bringing the csv into a pandas dataframe
features_data_df = pd.read_csv(features_file)
# convert date format
features_data_df['Date'] = pd.to_datetime(features_data_df['Date'])
# convert null / NaN values to zero
features_data_df.fillna(0, inplace=True)
# convert format of IsHoliday from bool to int
features_data_df['IsHoliday'] = features_data_df['IsHoliday'].apply(lambda x: 1 if x== True else 0)
# printing first five records in the dataframe
features_data_df.head()

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,2012-01-06,49.01,3.157,6277.39,21813.16,143.10,1450.13,8483.00,219.714258,7.348,0
1,1,2012-01-13,48.53,3.261,5183.29,8025.87,42.24,453.08,3719.38,219.892526,7.348,0
2,1,2012-01-20,54.11,3.268,4139.87,2807.19,33.88,500.62,3400.21,219.985689,7.348,0
3,1,2012-01-27,54.26,3.290,1164.46,1082.74,44.00,11.00,1222.19,220.078852,7.348,0
4,1,2012-02-03,56.55,3.360,34577.06,3579.21,160.53,32403.87,5630.40,220.172015,7.348,0


In [3]:
features_data_df.dtypes

Store                    int64
Date            datetime64[ns]
Temperature            float64
Fuel_Price             float64
MarkDown1              float64
MarkDown2              float64
MarkDown3              float64
MarkDown4              float64
MarkDown5              float64
CPI                    float64
Unemployment           float64
IsHoliday                int64
dtype: object

In [4]:
features_data_df.count()

Store           2340
Date            2340
Temperature     2340
Fuel_Price      2340
MarkDown1       2340
MarkDown2       2340
MarkDown3       2340
MarkDown4       2340
MarkDown5       2340
CPI             2340
Unemployment    2340
IsHoliday       2340
dtype: int64

In [4]:
# setting variable to reference the csv file
retail_sales_file = "sales data-set-1.csv"
# bringing the csv into a pandas dataframe
sales_data_df = pd.read_csv(retail_sales_file)
# convert date format
sales_data_df['Date'] = pd.to_datetime(sales_data_df['Date'])
# delete IsHoliday column because it is stored with features table
del sales_data_df['IsHoliday']
# printing first five records in the dataframe
sales_data_df.head()

,Store,Dept,Date,Weekly_Sales
0,1,1,2012-01-06,16065.49
1,2,1,2012-01-06,23076.47
2,3,1,2012-01-06,6755.83
3,4,1,2012-01-06,27095.99
4,5,1,2012-01-06,9442.66


In [5]:
sales_data_df.dtypes

Store                    int64
Dept                     int64
Date            datetime64[ns]
Weekly_Sales           float64
dtype: object

In [6]:
sales_data_df.count()

Store           127438
Dept            127438
Date            127438
Weekly_Sales    127438
dtype: int64

In [7]:
# setting variable to reference the csv file
stores_file = "stores data-set.csv"
# bringing the csv into a pandas dataframe
stores_data_df = pd.read_csv(stores_file)
# printing first five records in the dataframe
stores_data_df.head()

,Store,Type,Size
0,1,A,151315
1,2,A,202307
2,3,B,37392
3,4,A,205863
4,5,B,34875


In [8]:
stores_data_df.dtypes

Store     int64
Type     object
Size      int64
dtype: object

In [8]:
stores_data_df.count()

Store    45
Type     45
Size     45
dtype: int64

## Connect to mySQL and load tables

In [12]:
rds_connection_string = "root:<password>@localhost:3306/retail_db"
engine = create_engine(f'mysql://{rds_connection_string}')

In [10]:
# check for tables in the retail_db
engine.table_names()

['features', 'sales', 'stores']

In [21]:
features_data_df.to_sql(name='features', con=engine, if_exists='append', index=False)

In [11]:
sales_data_df.to_sql(name='sales', con=engine, if_exists='append', index=False, chunksize=10000)

In [11]:
stores_data_df.to_sql(name='stores', con=engine, if_exists='append', index=False)